In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/database_24_25.csv") #dataframe
df.head(10)
#Note: these are individual games

,Player,Tm,Opp,Res,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,Data
0,Jayson Tatum,BOS,NYK,W,30.30,14,18,0.778,8,11,...,4,4,10,1,1,1,1,37,38.1,2024-10-22
1,Anthony Davis,LAL,MIN,W,37.58,11,23,0.478,1,3,...,13,16,4,1,3,1,1,36,34.0,2024-10-22
2,Derrick White,BOS,NYK,W,26.63,8,13,0.615,6,10,...,3,3,4,1,0,0,1,24,22.4,2024-10-22
3,Jrue Holiday,BOS,NYK,W,30.52,7,9,0.778,4,6,...,2,4,4,1,0,0,2,18,19.5,2024-10-22
4,Miles McBride,NYK,BOS,L,25.85,8,10,0.800,4,5,...,0,0,2,0,0,1,1,22,17.8,2024-10-22
5,Rui Hachimura,LAL,MIN,W,35.08,7,14,0.500,1,4,...,2,5,1,2,1,0,2,18,15.9,2024-10-22
6,Jaylen Brown,BOS,NYK,W,29.90,7,18,0.389,5,9,...,5,7,1,1,0,1,3,23,15.6,2024-10-22
7,Rudy Gobert,MIN,LAL,L,35.33,5,8,0.625,0,0,...,11,14,2,0,1,1,4,13,13.9,2024-10-22
8,Julius Randle,MIN,LAL,L,34.32,5,10,0.500,1,3,...,6,9,4,0,0,2,3,16,13.7,2024-10-22
9,Al Horford,BOS,NYK,W,26.08,4,7,0.571,3,5,...,3,3,5,1,1,0,2,11,13.0,2024-10-22


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  16512 non-null  object 
 1   Tm      16512 non-null  object 
 2   Opp     16512 non-null  object 
 3   Res     16512 non-null  object 
 4   MP      16512 non-null  float64
 5   FG      16512 non-null  int64  
 6   FGA     16512 non-null  int64  
 7   FG%     16512 non-null  float64
 8   3P      16512 non-null  int64  
 9   3PA     16512 non-null  int64  
 10  3P%     16512 non-null  float64
 11  FT      16512 non-null  int64  
 12  FTA     16512 non-null  int64  
 13  FT%     16512 non-null  float64
 14  ORB     16512 non-null  int64  
 15  DRB     16512 non-null  int64  
 16  TRB     16512 non-null  int64  
 17  AST     16512 non-null  int64  
 18  STL     16512 non-null  int64  
 19  BLK     16512 non-null  int64  
 20  TOV     16512 non-null  int64  
 21  PF      16512 non-null  int64  
 22

In [15]:
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# --- Find back-to-backs at the team level ---
team_dates = (
    df[['Tm', 'Data']]
    .drop_duplicates()
    .sort_values(by=['Tm', 'Data'])
)

team_dates['days_since_last'] = team_dates.groupby('Tm')['Data'].diff().dt.days
team_dates['is_b2b_second'] = team_dates['days_since_last'] == 1

# Extract just the (Tm, Data) combos that are B2B second games
b2b_second_dates = team_dates.loc[team_dates['is_b2b_second'], ['Tm', 'Data']]

# --- Join back to player-level data ---
b2b_second_df = df.merge(b2b_second_dates, on=['Tm', 'Data'], how='inner')


In [16]:
# --- Display info ---
print(f"Original shape: {df.shape}")
print(f"Back-to-back second games shape: {b2b_second_df.shape}\n")

print("--- DataFrame Info ---")
print(b2b_second_df.info())

Original shape: (16512, 25)
Back-to-back second games shape: (2874, 25)

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2874 entries, 0 to 2873
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Player  2874 non-null   object        
 1   Tm      2874 non-null   object        
 2   Opp     2874 non-null   object        
 3   Res     2874 non-null   object        
 4   MP      2874 non-null   float64       
 5   FG      2874 non-null   int64         
 6   FGA     2874 non-null   int64         
 7   FG%     2874 non-null   float64       
 8   3P      2874 non-null   int64         
 9   3PA     2874 non-null   int64         
 10  3P%     2874 non-null   float64       
 11  FT      2874 non-null   int64         
 12  FTA     2874 non-null   int64         
 13  FT%     2874 non-null   float64       
 14  ORB     2874 non-null   int64         
 15  DRB     2874 non-null   int64         
 16  

In [17]:
# --- Compute per-player averages ---
player_stats_b2b = (
    b2b_second_df.groupby('Player', as_index=False)
    .agg({
        'PTS': 'mean',   # Points per game
        'AST': 'mean',   # Assists per game
        'TRB': 'mean',   # Total rebounds per game
        'STL': 'mean',   # Steals per game
        'BLK': 'mean',   # Blocks per game
        'TOV': 'mean',   # Turnovers per game
        'MP': 'mean',    # Minutes per game
        # 'GmSc': 'mean'   # Game Score
    })
    .rename(columns={
        'PTS': 'PPG',
        'AST': 'APG',
        'TRB': 'RPG',
        'STL': 'SPG',
        'BLK': 'BPG',
        'TOV': 'TOPG',
        'MP': 'MPG',
        # 'GmSc': 'GameScore'
    })
)

# Round for readability
player_stats_b2b = player_stats_b2b.round(2)

# --- Optional: sort by points or minutes ---
player_stats_b2b = player_stats_b2b.sort_values(by='PPG', ascending=False)

# --- Display summary ---
print(f"Number of players with B2B second game data: {len(player_stats_b2b)}")
print(player_stats_b2b.head(10))

# --- (Optional) save ---
# player_stats_b2b.to_csv("player_b2b_second_game_averages.csv", index=False)


Number of players with B2B second game data: 526
                      Player    PPG    APG    RPG   SPG   BPG  TOPG    MPG
170    Giannis Antetokounmpo  34.43   5.00  11.57  0.57  1.00  4.14  35.47
388             Nikola Jokić  34.33  10.56  11.56  1.56  0.56  3.00  35.54
451  Shai Gilgeous-Alexander  32.12   7.00   5.00  1.88  1.50  2.00  35.94
290       Karl-Anthony Towns  31.71   3.57  14.29  1.14  0.71  3.29  37.98
338              Luka Dončić  31.33  12.00   7.33  2.00  0.00  2.67  39.59
324             Kyrie Irving  31.17   4.50   3.67  0.83  0.17  2.17  39.07
29           Anthony Edwards  31.00   5.12   5.62  1.62  0.88  3.25  37.06
210            Jalen Brunson  30.57   8.00   2.43  0.86  0.14  1.71  36.22
326              LaMelo Ball  29.75   8.00   4.50  2.00  0.00  5.00  34.78
227             Jared McCain  29.00   6.50   4.50  1.00  0.00  3.50  38.40


In [18]:
# Make sure 'Data' is parsed as datetime if not already
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# --- Compute per-player averages for all games ---
player_stats_all = (
    df.groupby('Player', as_index=False)
    .agg({
        'PTS': 'mean',
        'AST': 'mean',
        'TRB': 'mean',
        'STL': 'mean',
        'BLK': 'mean',
        'TOV': 'mean',
        'MP': 'mean',
    })
    .rename(columns={
        'PTS': 'PPG',
        'AST': 'APG',
        'TRB': 'RPG',
        'STL': 'SPG',
        'BLK': 'BPG',
        'TOV': 'TOPG',
        'MP': 'MPG',
    })
)

# Round for readability
player_stats_all = player_stats_all.round(2)

# Sort by points for a quick view
player_stats_all = player_stats_all.sort_values(by='PPG', ascending=False)

# --- Display summary ---
print(f"Number of players in full dataset: {len(player_stats_all)}")
print(player_stats_all.head(10))

# --- (Optional) Save ---
#player_stats_all.to_csv("player_all_game_averages.csv", index=False)


Number of players in full dataset: 562
                      Player    PPG    APG    RPG   SPG   BPG  TOPG    MPG
480  Shai Gilgeous-Alexander  32.44   5.90   5.22  1.94  1.00  2.68  34.07
183    Giannis Antetokounmpo  31.62   5.98  12.33  0.79  1.36  3.45  34.95
414             Nikola Jokić  29.70  10.33  12.70  1.83  0.67  3.09  36.23
364              Luka Dončić  28.14   7.82   8.32  2.00  0.41  3.41  35.66
351              LaMelo Ball  28.03   7.38   5.22  1.34  0.31  3.69  33.36
535             Tyrese Maxey  27.84   6.20   3.43  1.93  0.39  2.48  38.03
31           Anthony Edwards  27.22   4.53   5.78  1.14  0.69  3.45  36.73
330             Kevin Durant  26.92   4.18   6.08  0.82  1.33  3.03  36.16
250        Jaren Jackson Jr.  26.56   2.94   6.39  1.00  1.44  2.33  32.11
263             Jayson Tatum  26.55   5.65   8.65  1.22  0.53  2.86  36.32


In [19]:
# Merge both DataFrames
comparison = pd.merge(
    player_stats_all,
    player_stats_b2b,
    on='Player',
    how='inner',
    suffixes=('_All', '_B2B2')
)

# --- Compute diffs only for common stats ---
stats = ['PPG', 'APG', 'RPG', 'SPG', 'BPG', 'TOPG', 'MPG', 'GameScore']
for stat in stats:
    if f'{stat}_All' in comparison.columns and f'{stat}_B2B2' in comparison.columns:
        comparison[f'{stat}_Diff'] = comparison[f'{stat}_B2B2'] - comparison[f'{stat}_All']

# Round & sort
comparison = comparison.round(2)
comparison = comparison.sort_values(by='PPG_Diff')

# print(comparison[['Player', 'PPG_All', 'PPG_B2B2', 'PPG_Diff']].head(10))


In [20]:
# Sort by points difference (biggest rise first)
ppg_risers = comparison.sort_values(by='PPG_Diff', ascending=False)

# Round for readability
ppg_risers = ppg_risers.round(2)

# Display top 15 players with the biggest PPG increases
print("\nTop 15 players with the biggest PPG increases in 2nd games of back-to-backs:")
print(
    ppg_risers[
        ['Player', 'PPG_All', 'PPG_B2B2', 'PPG_Diff', 'MPG_All', 'MPG_B2B2', 'MPG_Diff']
    ].head(15)
)


Top 15 players with the biggest PPG increases in 2nd games of back-to-backs:
                     Player  PPG_All  PPG_B2B2  PPG_Diff  MPG_All  MPG_B2B2  \
93             Jared McCain    15.30     29.00     13.70    25.73     38.40   
366  Dariq Miller-Whitehead     4.50     18.00     13.50     6.94     23.87   
194           GG Jackson II     9.89     21.00     11.11    17.60     25.90   
448              AJ Johnson     2.44     13.00     10.56     6.02     29.10   
486           Pacome Dadiet     1.46      9.00      7.54     6.39     17.87   
23       Karl-Anthony Towns    24.33     31.71      7.38    34.72     37.98   
208           Dalton Knecht     9.42     16.67      7.25    20.71     25.57   
138         Daeqwon Plowden    12.00     19.00      7.00    16.36     25.25   
22             Kyrie Irving    24.37     31.17      6.80    36.18     39.07   
354            Jaylen Clark     4.80     11.50      6.70    11.58     17.92   
175       Spencer Dinwiddie    10.30     17.00      6

In [21]:
# Sort by points difference (biggest drop first)
ppg_drops = comparison.sort_values(by='PPG_Diff', ascending=True)

# Round for readability
ppg_drops = ppg_drops.round(2)

# Display top 15 players with biggest PPG drops
print("\nTop 15 players with the biggest PPG drops in 2nd games of back-to-backs:")
print(
    ppg_drops[
        ['Player', 'PPG_All', 'PPG_B2B2', 'PPG_Diff', 'MPG_All', 'MPG_B2B2', 'MPG_Diff']
    ].head(15)
)


Top 15 players with the biggest PPG drops in 2nd games of back-to-backs:
                Player  PPG_All  PPG_B2B2  PPG_Diff  MPG_All  MPG_B2B2  \
43        Jordan Poole    20.84     12.29     -8.55    31.16     28.54   
29          Trae Young    23.25     16.00     -7.25    36.07     33.69   
282     Andre Drummond     6.90      0.00     -6.90    18.82     14.40   
283       Torrey Craig     6.89      0.00     -6.89    12.54      8.35   
256         Al Horford     7.85      1.00     -6.85    26.20     25.55   
12       Anthony Davis    25.74     19.17     -6.57    34.27     31.70   
13     Trey Murphy III    25.62     20.00     -5.62    37.05     37.67   
215      Miles McBride     9.15      3.80     -5.35    23.27     19.60   
244   Kevin Porter Jr.     8.22      3.00     -5.22    18.72     15.42   
21   Victor Wembanyama    24.42     19.25     -5.17    33.07     34.58   
134   Tim Hardaway Jr.    12.16      7.00     -5.16    30.81     30.08   
270     Wendell Carter     7.33      2

In [22]:
mpg_drops = comparison.sort_values(by='MPG_Diff', ascending=True)

# Round for readability
mpg_drops = mpg_drops.round(2)

# Display top 15 players with biggest MPG drops
print("\nTop 15 players with the biggest MPG drops in 2nd games of back-to-backs:")
print(
    mpg_drops[
        ['Player', 'MPG_All', 'MPG_B2B2', 'MPG_Diff', 'PPG_All', 'PPG_B2B2', 'PPG_Diff']
    ].head(15)
)



Top 15 players with the biggest MPG drops in 2nd games of back-to-backs:
               Player  MPG_All  MPG_B2B2  MPG_Diff  PPG_All  PPG_B2B2  \
287     Elfrid Payton    20.54     14.18     -6.36     6.71      5.50   
391     Mohamed Bamba    12.84      6.59     -6.25     3.81      1.50   
355    Trevelin Queen    15.23      9.51     -5.72     4.74      2.60   
378        Kyle Lowry    19.29     14.39     -4.90     4.16      3.33   
179  Jaime Jaquez Jr.    25.21     20.46     -4.75    10.26      9.00   
58      Jalen Johnson    35.67     30.96     -4.71    18.89     17.83   
339      Gabe Vincent    20.31     15.66     -4.65     5.38      1.75   
282    Andre Drummond    18.82     14.40     -4.42     6.90      0.00   
171  Dereck Lively II    23.57     19.33     -4.24    10.71      7.00   
237       Kel'el Ware    18.36     14.15     -4.21     8.43      7.14   
383   Josh Richardson    18.69     14.48     -4.21     4.00      4.00   
283      Torrey Craig    12.54      8.35     -4.19

In [23]:
import matplotlib.pyplot as plt
#TODO: Visualizations